# 1. Self 손글씨 맞추기 실습

직접 작성한 숫자 이미지 + MNIST 데이터셋을 사용해 학습 예측하는 코드를 작성 

MNIST 데이터셋은 손으로 쓴 숫자(0-9) 이미지들로 구성된 데이터셋

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from PIL import Image  # 사용자가 제공한 이미지를 처리하기 위해 사용

# MNIST 데이터셋 로드
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 데이터 전처리
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# 간단한 CNN 모델 생성
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

# 사용자 이미지 예측 함수 정의
def predict_image(image_path):
    # 이미지 로드 및 전처리
    img = Image.open(image_path).convert('L')
    img = img.resize((28, 28))
    img = np.array(img).astype('float32') / 255
    img = 1 - img  # 흑백 반전 (흰 배경에 검은 숫자로 맞추기 위해)
    img = img.reshape((1, 28, 28, 1))
    
    # 예측
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    print(f"예측된 숫자: {predicted_label}")

# 테스트할 사용자 이미지 경로
image_path = 'KakaoTalk_20241124_215514801.jpg'  # 여기서 'your_image.png'를 예측할 이미지 파일 경로로 변경하세요
predict_image(image_path)


C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - accuracy: 0.8576 - loss: 0.4543 - val_accuracy: 0.9851 - val_loss: 0.0463
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.9820 - loss: 0.0549 - val_accuracy: 0.9894 - val_loss: 0.0296
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - accuracy: 0.9896 - loss: 0.0337 - val_accuracy: 0.9888 - val_loss: 0.0327
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9908 - loss: 0.0283 - val_accuracy: 0.9881 - val_loss: 0.0379
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.9927 - loss: 0.0239 - val_accuracy: 0.9892 - val_loss: 0.0338
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
예측된 숫자: 5


손글씨 코드설명

#### 1)  라이브러리 임포트

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras import layers, models
# from tensorflow.keras.datasets import mnist
# from tensorflow.keras.utils import to_categorical
# from PIL import Image

In [ ]:
# PIL (Python Imaging Library)는 이미지를 처리하는 데 사용되는 라이브러리
# 현재는 유지보수가 중단되어 대신 Pillow라는 이름으로 발전되었습니다. 
# Pillow는 PIL의 호환성을 유지하면서 추가 기능과 버그 수정을 포함한 라이브러리

#### 2) MNIST 데이터셋 로드

MNIST 데이터셋은 손으로 쓴 숫자 이미지 (0-9)로 구성되어 있으며, Keras에서 쉽게 로드할 수 있습니다.

(train_images, train_labels)는 학습용 데이터, (test_images, test_labels)는 테스트용 데이터를 의미합니다.

In [ ]:
#(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


#### 3) 데이터 전처리

In [ ]:
# 1. train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
# train_images: 훈련 데이터셋에 포함된 이미지 배열입니다. 
# 보통 MNIST의 경우 (60000, 28, 28) 형태로 6만 개의 28x28 픽셀 그레이스케일 이미지로 구성됩니다.
# reshape((60000, 28, 28, 1)):  
# 배열을 (60000, 28, 28, 1) 형태로 변경합니다.
# 여기서 28, 28은 이미지의 높이와 너비를 의미하고, 1은 채널 수를 의미합니다. 이는 그레이스케일 이미지이기 때문에 채널이 1입니다. 컬러 이미지의 경우에는 채널이 3 (RGB)입니다.
# 이 형태로 변환하는 것은 CNN (합성곱 신경망) 모델에 데이터를 전달할 때 필요한 형태이기 때문입니다. CNN은 이미지의 공간적 차원을 고려하기 때문에 채널 정보를 명시적으로 지정해 주어야 합니다.

# CNN 모델의 입력 형식
# CNN은 이미지의 공간적 정보(즉, 높이, 너비, 채널)를 고려하여 특징을 추출하는 구조입니다. 
# CNN이 이미지 데이터를 처리할 때 각 픽셀 간의 관계와 지역적인 패턴을 이해하기 위해 4차원 텐서를 입력으로 받습니다. 일반적으로 CNN의 입력 텐서는 다음과 같은 형식을 가집니다:
# (배치 크기, 높이, 너비, 채널 수)
# 이 형식에서 각 차원의 의미는 다음과 같습니다:
# 배치 크기 (60000): 훈련 데이터의 이미지 개수입니다. 여기서는 60,000개의 이미지를 한 번에 처리합니다.
# 높이와 너비 (28, 28): 각 이미지의 높이와 너비입니다. MNIST 데이터셋의 경우, 각 이미지는 28x28 픽셀 크기를 가집니다.
# 채널 수 (1): 이미지의 채널 수를 나타냅니다. 여기서는 1로, 이는 그레이스케일 이미지임을 의미합니다. 만약 컬러 이미지라면 채널 수는 3(RGB)일 것입니다.
# 이렇게 (60000, 28, 28, 1) 형태로 변경하면, CNN이 데이터를 적절하게 해석하고 처리할 수 있는 형태가 됩니다. CNN은 이 4차원 텐서를 사용하여 각 필터를 적용하고, 공간적 특성을 학습합니다.

# astype('float32'): 
# 이미지의 데이터를 float32 형식으로 변환합니다. 원래 train_images는 일반적으로 0~255의 정수 값을 가지는데, 신경망의 계산에 적합하도록 float32로 변환합니다.
# / 255:
# 이미지를 정규화하는 과정입니다. 이미지의 픽셀 값이 0~255 사이의 정수인 경우, / 255를 통해 모든 값을 01 사이로 정규화합니다.
# 이 정규화는 신경망의 훈련을 더 안정적이고 빠르게 만들며, 가중치 업데이트가 더 효과적으로 이루어지도록 합니다.

# 2. test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
# 테스트 데이터셋에 대해 동일한 전처리를 수행합니다.
# **test_images**는 테스트 이미지로 구성된 배열이며, reshape, astype, / 255로 훈련 이미지와 같은 방식으로 전처리합니다.
# 테스트 데이터의 형태는 (10000, 28, 28)이었으며, 이를 (10000, 28, 28, 1)로 변환합니다. 이 데이터는 평가나 예측에 사용됩니다.

# 3. train_labels = to_categorical(train_labels)
# train_labels: 훈련 데이터셋의 레이블입니다. 원래는 각 이미지의 클래스를 나타내는 정수 배열로 구성됩니다. 예를 들어, MNIST 데이터셋의 경우 각 레이블은 0에서 9 사이의 정수로 되어 있습니다.
# to_categorical(train_labels):
# 레이블을 원-핫 인코딩(one-hot encoding) 형태로 변환합니다.
# 예를 들어, 레이블 3은 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]으로 변환됩니다. 이 형식은 딥러닝 모델에서 분류 문제를 다룰 때 더 적합합니다.
# 원-핫 인코딩은 모델이 각 클래스의 확률을 출력하도록 학습하기 위해 필요합니다.

# 4. test_labels = to_categorical(test_labels)
# 테스트 레이블에 대해서도 동일하게 원-핫 인코딩을 적용합니다.

#### 4) CNN 모델 생성

In [ ]:
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# model = models.Sequential()
# Sequential 모델을 정의합니다. 

# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# Conv2D 레이어: 합성곱 레이어입니다.
# 32: 필터의 개수를 의미합니다. 이 레이어는 32개의 필터(커널)를 사용하여 입력 이미지에서 특징을 추출합니다.
# (3, 3): 필터의 크기입니다. 이 경우, 각 필터는 3x3 크기를 가지며, 입력 이미지에 적용됩니다.
# activation='relu': ReLU (Rectified Linear Unit) -->모델이 복잡한 패턴을 학습할 수 있도록 돕습니다.
# input_shape=(28, 28, 1): 입력 데이터의 형태를 정의합니다. (28, 28, 1)은 높이가 28, 너비가 28, 채널이 1인 그레이스케일 이미지를 의미합니다. 첫 번째 레이어에서만 입력의 크기를 명시해줍니다.

# model.add(layers.MaxPooling2D((2, 2)))
# axPooling2D 레이어: 최대 풀링 레이어입니다.
#  (2, 2): 풀링 크기를 나타냅니다. 이 레이어는 입력을 (2, 2) 영역으로 나누고, 각 영역에서 최대값을 취합니다. 이 과정을 통해 특징 맵의 크기를 줄이고 계산량을 감소시키며, 주요 특징을 더 잘 추출하도록 돕습니다.

# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# 또 다른 Conv2D 레이어를 추가합니다.
# 64: 이번 레이어에서는 64개의 필터를 사용합니다. 필터 개수를 늘리면서 더 복잡하고 추상적인 특징을 추출합니다.
# (3, 3) 필터 크기와 **activation='relu'**는 이전과 동일합니다.

# model.add(layers.MaxPooling2D((2, 2)))
# MaxPooling2D 레이어를 다시 추가하여, 특징 맵의 크기를 더 줄이고 중요한 특징을 추출합니다.

# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# 또 다른 Conv2D 레이어로 깊이를 더 쌓습니다.
# 이 레이어는 이미지에서 더 높은 수준의 특징을 추출하는 역할을 합니다. 특징 추출 단계가 깊어질수록 모델이 데이터의 복잡한 구조와 패턴을 더 잘 학습하게 됩니다.

# model.add(layers.Flatten())
# Flatten 레이어: 2D 특징 맵을 1D 벡터로 펼칩니다.
# 이 단계는 합성곱 레이어와 완전 연결 레이어(Dense 레이어)를 연결하기 위해 필요합니다. CNN에서 추출한 모든 특징을 하나의 긴 벡터로 변환하여 다음 레이어에 전달합니다.

# model.add(layers.Dense(64, activation='relu'))
# Dense 레이어 (완전 연결 레이어):
# 64: 64개의 뉴런을 가진 레이어를 추가합니다.
# activation='relu': ReLU 활성화 함수를 사용하여 비선형성을 추가합니다.
# 이 레이어는 추출된 특징을 바탕으로 특정 패턴을 학습하는 역할을 합니다.

# model.add(layers.Dense(10, activation='softmax'))
# Dense 레이어 (출력 레이어):
# 10: 10개의 뉴런을 가진 레이어로, 출력 차원이 10임을 의미합니다. 이는 보통 분류할 클래스의 개수와 일치합니다. 예를 들어 MNIST 데이터셋에서는 숫자 0부터 9까지 총 10개의 클래스를 분류하기 때문에 출력 레이어에 10개의 뉴런을 사용합니다.
# activation='softmax': Softmax 활성화 함수는 각 클래스에 속할 확률을 출력하도록 합니다. 출력된 10개의 값은 모두 0과 1 사이의 값을 가지며, 합이 1이 되도록 정규화됩니다. 이는 다중 클래스 분류 문제에 적합한 활성화 함수입니다.

# 전체 모델 요약
# Conv2D & MaxPooling2D 레이어들: 입력 이미지에서 점점 더 높은 수준의 특징을 추출합니다. 필터 개수를 늘리고, 풀링 레이어를 통해 공간적 차원을 줄이며 특징의 중요도를 높입니다.
# Flatten 레이어: 다차원의 특징 맵을 1차원 벡터로 변환하여, 완전 연결 레이어와 연결합니다.
# Dense 레이어: 추출된 특징을 바탕으로 학습하여 분류 작업을 수행합니다. 마지막 레이어에서는 softmax 활성화 함수를 사용하여 클래스별 확률을 출력합니다.

#### 5) 모델 컴파일

In [ ]:
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
# adam 옵티마이저를 사용해 모델을 컴파일합니다.
# 손실 함수로는 categorical_crossentropy를 사용하며, 이는 다중 클래스 분류에 적합합니다.

In [ ]:
# 1. optimizer='adam'
# 옵티마이저(Optimizer) : 모델의 가중치를 학습하고 업데이트하는 방법을 결정합니다.
# adam( Adaptive Moment Estimation) : 딥러닝에서 널리 사용되는 적응형 학습률(adaptive learning rate)을 가진 옵티마이저입니다.
# adam 옵티마이저는 모멘텀(momentum)과 RMSProp의 장점을 결합한 방식으로, 학습 속도를 자동으로 조정하면서 빠르고 안정적으로 수렴할 수 있도록 도와줍니다.

# 2. loss='categorical_crossentropy'
# 손실 함수(Loss Function) : 모델이 얼마나 잘 예측하고 있는지를 측정하는 함수로, 모델 학습의 목표는 이 손실 값을 최소화하는 것입니다.
# categorical_crossentropy : 다중 클래스 분류 문제에 사용되는 손실 함수입니다.
# 이 함수는 실제 레이블(원-핫 인코딩된 벡터)과 모델의 예측 확률 간의 차이를 계산합니다.
# 예를 들어, MNIST와 같은 문제에서 0부터 9까지 10개의 클래스를 분류하는 경우, 실제 레이블과 모델의 예측 확률 분포 간의 차이를 줄이기 위해 categorical_crossentropy를 사용합니다.
# Cross-Entropy는 각 클래스에 대해 모델의 확률 예측 값이 실제 레이블과 얼마나 잘 맞는지를 평가합니다. 값이 작을수록 모델의 예측이 더 정확하다는 의미입니다.

# 3. metrics=['accuracy']
# 평가지표(Metrics) : 모델의 성능을 평가하는 기준을 정의합니다.
# accuracy: 훈련 중에 정확도(Accuracy)를 모니터링합니다. 이는 모델이 올바른 클래스를 얼마나 자주 예측하는지를 나타냅니다.
# 예를 들어, 정확도가 0.90(90%)이면, 전체 데이터 중 90%의 예측이 정확하다는 것을 의미합니다.

#### 6) 모델 학습

In [ ]:
# model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

# fit() 함수를 통해 모델을 학습시킵니다.
# 에포크 수는 5회, 배치 크기는 64로 설정하였습니다. validation_data는 학습 동안 모델의 성능을 평가하기 위해 사용됩니다.

In [ ]:
# 1. train_images와 train_labels
# train_images: 훈련에 사용되는 입력 이미지 데이터입니다. 여기서 train_images는 전처리된 이미지 데이터로, 일반적으로 (60000, 28, 28, 1) 형태의 4차원 텐서입니다.
# train_labels: 훈련에 사용되는 정답 레이블입니다. 이 레이블은 원-핫 인코딩된 형태로, 각 이미지에 해당하는 클래스(예: 숫자 0부터 9까지)를 나타냅니다.
# 이 데이터를 사용하여 모델은 입력 이미지와 정답 레이블을 통해 학습을 진행하게 됩니다.

# 2. epochs=5
# **에포크(Epoch)**는 전체 훈련 데이터셋을 모델이 한 번 학습하는 과정을 의미합니다.
# 에포크 수가 증가하면 모델이 데이터를 더 많이 학습할 수 있지만, 너무 많은 에포크는 과적합(overfitting)을 유발할 수 있습니다. 과적합은 모델이 훈련 데이터에 지나치게 맞춰져 일반화 성능이 떨어지는 현상을 의미합니다.

# 3. batch_size=64
# 배치 크기(Batch Size)는 모델이 한 번에 처리할 데이터의 개수를 나타냅니다.
# batch_size=64는 한 번의 업데이트마다 64개의 이미지를 사용하겠다는 의미입니다.
# 모델은 전체 훈련 데이터를 64개씩 나누어 학습을 진행하며, 배치 단위로 가중치를 업데이트합니다.
# 배치 크기가 작은 경우: 학습이 더 세밀하게 이루어져서 빠르게 수렴할 수 있지만, 계산 비용이 높아질 수 있습니다.
# 배치 크기가 큰 경우: 계산이 더 효율적이고 안정적일 수 있지만, 학습이 느려질 수 있습니다.
# 배치 크기를 적절히 설정하는 것은 학습의 효율성과 모델의 성능에 중요한 영향을 미칩니다.

# 4. validation_data=(test_images, test_labels)
# validation_data: 검증 데이터셋으로, 모델의 성능을 평가하기 위해 훈련 중에 사용됩니다.
# test_images: 검증에 사용되는 이미지 데이터로, (10000, 28, 28, 1) 형태의 4차원 텐서입니다.
# test_labels: 검증에 사용되는 레이블로, 원-핫 인코딩된 형태의 정답 레이블입니다.
# 검증 데이터(validation data)는 훈련 데이터와는 독립적인 데이터로 모델의 일반화 성능을 평가하는 데 사용됩니다.
# 매 에포크가 끝날 때마다 검증 데이터에 대해 모델을 평가하여 훈련 과정에서의 성능 변화와 과적합 여부를 모니터링할 수 있습니다.

# model.fit()의 전체적인 역할
# model.fit()은 모델을 훈련시키는 메서드로, 훈련 데이터와 레이블을 사용하여 모델의 가중치를 조정합니다.
# 모델이 훈련 데이터를 학습하면서 손실(loss)을 줄이고, 주어진 에포크 수 만큼 반복하여 모델의 성능을 점차 향상시킵니다.
# 검증 데이터를 통해 각 에포크가 끝날 때마다 모델이 잘 학습되고 있는지, 과적합이 발생하고 있는지를 모니터링할 수 있습니다.

#### 7) 사용자 이미지 예측 함수 정의

In [ ]:
# def predict_image(image_path):
#     # 이미지 로드 및 전처리
#     img = Image.open(image_path).convert('L')
#     img = img.resize((28, 28))
#     img = np.array(img).astype('float32') / 255
#     img = 1 - img  # 흑백 반전 (흰 배경에 검은 숫자로 맞추기 위해)
#     img = img.reshape((1, 28, 28, 1))
    
#     # 예측
#     prediction = model.predict(img)
#     predicted_label = np.argmax(prediction)
#     print(f"예측된 숫자: {predicted_label}")

In [ ]:
# def predict_image(image_path):
# 함수 정의: predict_image(image_path)는 이미지 파일 경로를 입력으로 받아 그 이미지가 나타내는 숫자를 예측하는 함수입니다.
# image_path: 이미지 파일의 경로를 받습니다.

# 1. 이미지 로드 및 전처리
#     img = Image.open(image_path).convert('L')
# 이미지 로드: Image.open(image_path)를 사용하여 지정된 경로에 있는 이미지를 로드합니다.
# 그레이스케일 변환 (convert('L')): 이미지를 그레이스케일로 변환합니다. 'L'은 흑백 모드를 의미하며, 픽셀 값을 0에서 255 사이의 회색조로 나타냅니다.
#     img = img.resize((28, 28))
# 이미지 크기 조정: 이미지를 (28, 28) 크기로 조정합니다. 이 모델은 MNIST 데이터셋과 같은 28x28 픽셀 크기의 이미지로 훈련되었으므로, 예측을 위해 입력 이미지의 크기를 동일하게 맞춰야 합니다.
#    img = np.array(img).astype('float32') / 255
# 이미지 배열 변환: img를 넘파이 배열로 변환합니다. 이미지 데이터는 (28, 28)의 2차원 배열로 표현됩니다.
# 정규화 (/ 255): 배열 값을 0~1 사이의 값으로 정규화합니다. 원래 이미지의 픽셀 값은 0~255 사이의 정수 값이므로, 이를 / 255로 나누어 모든 값을 0과 1 사이로 변환합니다. 이는 모델의 훈련 데이터 전처리와 동일하게, 학습된 가중치에 맞추어 일관된 입력 형태를 제공합니다.
#    img = 1 - img  # 흑백 반전 (흰 배경에 검은 숫자로 맞추기 위해)
# 흑백 반전: 1 - img를 수행하여 이미지의 색상을 반전시킵니다.
# 일반적으로 MNIST 데이터셋의 이미지는 흰색 배경에 검은색 숫자로 되어 있습니다.
# 하지만 사용자로부터 입력받은 이미지는 검은색 배경에 흰색 숫자로 되어 있을 수 있습니다. 따라서 숫자를 검은색으로 바꾸기 위해 색상을 반전시킵니다.
#   img = img.reshape((1, 28, 28, 1))
# 배열 차원 변경: img를 (1, 28, 28, 1)로 재구성(reshape) 합니다.
# 1: 첫 번째 차원은 배치 크기입니다. 모델이 이미지를 배치 단위로 처리하기 때문에, 여기서는 입력 이미지를 하나의 배치로 간주합니다.
# 28, 28: 이미지의 높이와 너비입니다.
# 1: 채널 수입니다. 그레이스케일 이미지이므로 채널 수는 1입니다.
# 이렇게 4차원 텐서 (1, 28, 28, 1)로 만들어야 모델이 입력을 올바르게 처리할 수 있습니다.

# 2. 예측
#  prediction = model.predict(img)
# 모델 예측: 훈련된 모델(model)을 사용하여 전처리된 이미지(img)에 대한 예측을 수행합니다.
# model.predict(img) :  각 클래스(숫자 0부터 9까지)에 대해 확률 값을 반환하는 배열입니다. 예를 들어 [0.1, 0.05, 0.8, ...]와 같이 각 숫자에 대한 확률이 출력됩니다.
#  predicted_label = np.argmax(prediction)
# 예측된 레이블 찾기 (np.argmax): 반환된 예측 배열에서 최대 확률을 가진 인덱스를 찾습니다.
# **np.argmax(prediction)**은 예측 확률 값 중 가장 큰 값의 인덱스를 반환합니다. 예를 들어, 2번째 위치의 확률이 가장 높다면 2를 반환합니다. 이는 모델이 이미지가 숫자 2일 가능성이 가장 높다고 예측한 것입니다.
# print(f"예측된 숫자: {predicted_label}")
# 결과 출력: 예측된 숫자를 출력합니다. f"예측된 숫자: {predicted_label}"를 통해 예측된 결과를 사용자에게 보여줍니다.

#### 8) 이미지 경로 설정 및 예측 실행

In [ ]:
# image_path = 'your_image.png'  # 여기서 'your_image.png'를 예측할 이미지 파일 경로로 변경하세요
# predict_image(image_path)

In [ ]:
# 'your_image.png'를 사용자가 예측하고자 하는 이미지 파일 경로로 변경한 후, predict_image() 함수를 호출해 예측 결과를 출력합니다.

# 2. 0부터 9까지의 확률로 보여주가 : epochs=5

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from PIL import Image
import os

# MNIST 데이터셋 로드
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 데이터 전처리
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# 간단한 CNN 모델 생성
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

# 사용자 이미지 예측 함수 정의
def predict_image(image_path):
    if not os.path.exists(image_path):
        print(f"Error: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {image_path}")
        return
    
    # 이미지 로드 및 전처리
    img = Image.open(image_path).convert('L')
    img = img.resize((28, 28))
    img = np.array(img).astype('float32') / 255
    img = 1 - img  # 흑백 반전 (흰 배경에 검은 숫자로 맞추기 위해)
    img = img.reshape((1, 28, 28, 1))
    
    # 예측
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    print(f"예측된 숫자: {predicted_label}")
    print("0부터 9까지의 확률:")
    for i, prob in enumerate(prediction[0]):
        print(f"{i}: {prob * 100:.2f}%")

# 테스트할 사용자 이미지 경로
image_path = 'KakaoTalk_20241124_215514801.jpg'  # 여기서 'your_image.png'를 예측할 이미지 파일 경로로 변경하세요
predict_image(image_path)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step - accuracy: 0.8721 - loss: 0.4100 - val_accuracy: 0.9796 - val_loss: 0.0604
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9825 - loss: 0.0559 - val_accuracy: 0.9877 - val_loss: 0.0405
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.9884 - loss: 0.0366 - val_accuracy: 0.9872 - val_loss: 0.0419
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.9917 - loss: 0.0268 - val_accuracy: 0.9902 - val_loss: 0.0309
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9937 - loss: 0.0205 - val_accuracy: 0.9893 - val_loss: 0.0341
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
예측된 숫자: 7
0부터 9까지의 확률:
0: 7.08%
1: 13.14%
2: 10.42%
3: 6.38%
4: 8.49%
5: 9.13%
6: 5.68%
7: 30.87%
8: 3.04%
9: 5.75%


# 3. 0부터 9까지의 확률로 보여주가 : epochs=10

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from PIL import Image
import os

# MNIST 데이터셋 로드
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 데이터 전처리
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# 간단한 CNN 모델 생성
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_data=(test_images, test_labels))

# 사용자 이미지 예측 함수 정의
def predict_image(image_path):
    if not os.path.exists(image_path):
        print(f"Error: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {image_path}")
        return
    
    # 이미지 로드 및 전처리
    img = Image.open(image_path).convert('L')
    img = img.resize((28, 28))
    img = np.array(img).astype('float32') / 255
    img = 1 - img  # 흑백 반전 (흰 배경에 검은 숫자로 맞추기 위해)
    img = img.reshape((1, 28, 28, 1))
    
    # 예측
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    print(f"예측된 숫자: {predicted_label}")
    print("0부터 9까지의 확률:")
    for i, prob in enumerate(prediction[0]):
        print(f"{i}: {prob * 100:.2f}%")

# 테스트할 사용자 이미지 경로
image_path = 'KakaoTalk_20241124_215514801.jpg'  # 여기서 'your_image.png'를 예측할 이미지 파일 경로로 변경하세요
predict_image(image_path)


Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.8649 - loss: 0.4329 - val_accuracy: 0.9844 - val_loss: 0.0441
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.9826 - loss: 0.0560 - val_accuracy: 0.9876 - val_loss: 0.0429
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9878 - loss: 0.0398 - val_accuracy: 0.9879 - val_loss: 0.0349
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9905 - loss: 0.0297 - val_accuracy: 0.9896 - val_loss: 0.0308
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9936 - loss: 0.0210 - val_accuracy: 0.9910 - val_loss: 0.0293
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9942 - loss: 0.0177 - val_accuracy: 0.9913 - val_loss: 0.0273
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9948 - loss: 0.0161 - val_accuracy: 0.9898 - val_loss: 0.0314
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.9959 - loss: 0.0129 - 